In [124]:
import os
import csv
import pandas as pd
import numpy as np
from sklearn import linear_model,decomposition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from numpy import genfromtxt
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


In [125]:
data = genfromtxt('ACME-HappinessSurvey2020.csv', delimiter=',')
#print(data.shape)
#print(data)

In [126]:
new_data=data[1:len(data),:]
#print(new_data)
#print(new_data.shape)
X_F=new_data[:,1:len(data)]
#x=new_data[:,1:len(data)]
x=np.column_stack((new_data[:, 0],new_data[:, 2],new_data[:, 4]))
y=new_data[:,0]
#print(x)
#print(x.shape)
#print(y)
x_new=normalize(x)
sc=StandardScaler()
pca=decomposition.PCA()
logistic=linear_model.LogisticRegression()
pipe=Pipeline(steps=[('sc',sc),('pca',pca),('logistic',logistic)])

In [127]:
n_components=list(range(1,x_new.shape[1]+1,1))
C=np.logspace(-1,4,50)
#print(C)
penalty=['l2']
parameters=dict(pca__n_components=n_components,logistic__C=C,logistic__penalty=penalty)
clf=GridSearchCV(pipe,parameters)
clf.fit(x_new,y)

GridSearchCV(estimator=Pipeline(steps=[('sc', StandardScaler()), ('pca', PCA()),
                                       ('logistic', LogisticRegression())]),
             param_grid={'logistic__C': array([1.00000000e-01, 1.26485522e-01, 1.59985872e-01, 2.02358965e-01,
       2.55954792e-01, 3.23745754e-01, 4.09491506e-01, 5.17947468e-01,
       6.55128557e-01, 8.28642773e-01, 1.04811313e+00, 1.32571137e+00,
       1.67683294e+00, 2.1209508...
       7.19685673e+01, 9.10298178e+01, 1.15139540e+02, 1.45634848e+02,
       1.84206997e+02, 2.32995181e+02, 2.94705170e+02, 3.72759372e+02,
       4.71486636e+02, 5.96362332e+02, 7.54312006e+02, 9.54095476e+02,
       1.20679264e+03, 1.52641797e+03, 1.93069773e+03, 2.44205309e+03,
       3.08884360e+03, 3.90693994e+03, 4.94171336e+03, 6.25055193e+03,
       7.90604321e+03, 1.00000000e+04]),
                         'logistic__penalty': ['l2'],
                         'pca__n_components': [1, 2, 3]})

In [128]:
print('Best Penalty:', clf.best_estimator_.get_params()['logistic__penalty'])
print('Best C:', clf.best_estimator_.get_params()['logistic__C'])
print('Best Number Of Components:', clf.best_estimator_.get_params()['pca__n_components'])
print(); print(clf.best_estimator_.get_params()['logistic'])

Best Penalty: l2
Best C: 0.1
Best Number Of Components: 2

LogisticRegression(C=0.1)


In [129]:
# LOGISTIC REGRESSION TEST
cv_result=cross_val_score(clf,x_new,y,cv=4,n_jobs=-1)
print();print(cv_result)
print();print(cv_result.mean())
print();print(cv_result.std())


[1. 1. 1. 1.]

1.0

0.0


In [130]:
# SVM TEST
clf1 = SVC()
clf1.fit(x_new, y)
cv_result1=cross_val_score(clf1,x_new,y,cv=4,n_jobs=-1)

In [131]:
print();print(cv_result1)
print();print(cv_result1.mean())
print();print(cv_result1.std())


[1.         1.         1.         0.96774194]

0.9919354838709677

0.013968151673942547


In [132]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

In [133]:
for i in range(4):
    # Separate data into test and training sets
    # Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    clf2=svclassifier.fit(x_new, y)# Make prediction 
    cv_result2=cross_val_score(clf2,x_new,y,cv=4,n_jobs=-1)
    print("Evaluation:", kernels[i], "kernel")
    print();print(cv_result2)
    print();print(cv_result2.mean())
    print();print(cv_result2.std())
   

Evaluation: Polynomial kernel

[0.53125   0.5625    0.5483871 0.5483871]

0.5476310483870968

0.011074381321935398
Evaluation: RBF kernel

[1.         1.         1.         0.96774194]

0.9919354838709677

0.013968151673942547
Evaluation: Sigmoid kernel

[0.53125   0.5625    0.5483871 0.5483871]

0.5476310483870968

0.011074381321935398
Evaluation: Linear kernel

[1. 1. 1. 1.]

1.0

0.0


In [134]:
# FEATURES SELECTION
bestfeatures = SelectKBest(score_func=chi2, k=3)
fit = bestfeatures.fit(X_F,y)
dfscores = pd.DataFrame(fit.scores_)
print(dfscores)


          0
0  1.449041
1  0.036161
2  0.900104
3  0.106194
4  2.273286
5  0.541070


In [135]:
# Train Decision Tree Classifer

clf3 = DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf3 = clf3.fit(x_new,y)
cv_result3=cross_val_score(clf3,x_new,y,cv=4,n_jobs=-1)
print();print(cv_result3)
print();print(cv_result3.mean())
print();print(cv_result3.std())
   


[1. 1. 1. 1.]

1.0

0.0


In [136]:
# K-NN TEST
neigh = KNeighborsClassifier(n_neighbors=3)
neigh=neigh.fit(x_new, y)
cv_result4=cross_val_score(neigh,x_new,y,cv=4,n_jobs=-1)
print();print(cv_result4)
print();print(cv_result4.mean())
print();print(cv_result4.std())


[1.         1.         1.         0.96774194]

0.9919354838709677

0.013968151673942547


In [137]:
#NAVY BAYES TEST
gnb = GaussianNB()
y_pred = gnb.fit(x_new, y)
cv_result5=cross_val_score(y_pred,x_new,y,cv=4,n_jobs=-1)
print();print(cv_result5)
print();print(cv_result5.mean())
print();print(cv_result5.std())


[1. 1. 1. 1.]

1.0

0.0
